# Stencil Cython Serial

In [2]:
! cython --version

Cython version 0.29.20


In [2]:
! ifort --version

ifort (IFORT) 19.0.3.199 20190206
Copyright (C) 1985-2019 Intel Corporation.  All rights reserved.



In [3]:
! icc --version

icc (ICC) 19.0.3.199 20190206
Copyright (C) 1985-2019 Intel Corporation.  All rights reserved.



## Cython sem otimizações

### Testes

In [1]:
%reload_ext Cython

In [2]:
%%cython --force --compile-args=-O3
#cython: language_level=3
import sys
import time
import numpy as np

# inicialização de variáveis
n = 0           # nxn grid
energy = 0      # energy to be injected per iteration
niters = 0      # number of iterations

# demais variáveis
iters = i = j = size = sizeStart = sizeEnd = x = y = 0  # int(4)
t = t1 = tinit = tstencil = tenergy = tresto = 0.0      # float(8)
heat = 0.0      # float(8); calor total no sistema

# cria uma matriz sem inicializá-la
# 'F': store multi-dimensional data column-major (Fortran-style) \
#      order in memory
nsources = 3
sources = np.empty((nsources,2), np.int32, 'F')

# Lê os argumentos da linha de comando
#n = int(sys.argv[1])
#energy = float(sys.argv[2])
#niters = int(sys.argv[3])
#if niters % 2 != 0 :
#    print("iteration number should be even; it is ",niters)

# define os argumentos como parâmetros fixos, sem lê-los da \
# linha de comando
# 4800, 1, 500 = 1500; 100, 1, 10 = 30
n       = 100
energy  = 1
niters  = 10

t    = time.time()
size        = n + 2
sizeStart   = 2
sizeEnd     = n + 1

# cria e inicializa as matrizes com zeros
aold = np.zeros((size, size), np.double, 'F')
anew = np.zeros((size, size), np.double, 'F')

# inicializa as 3 fontes de calor
sources[:,:] =[ [n/2, n/2], [n/3, n/3], [n*4/5, n*8/9] ]

for iters in range(0, niters, 2) :

    ### iteracao impar: anew <- stencil(aold) ###

    # indices em C começam em 0, e em fortran começam em 1
    # range em python termina antes do fim, é "<" e não "<="
    for j in range(sizeStart-1, sizeEnd) :
        for i in range(sizeStart-1, sizeEnd) :
            anew[i,j] = aold[i,j] / 2.0 +  \
                        (  aold[i-1,j] + aold[i+1,j] +  \
                        aold[i,j-1] + aold[i,j+1]   ) / 4.0 / 2.0

    # indices precisam ser subtraídos de 1 porque em python começam \
    # em 0
    for i in range(0, nsources) :
        x = sources[i,0]
        y = sources[i,1]
        anew[x, y] = anew[x, y] + energy

    ### iteracao par: aold <- stencil(anew) ###

    for j in range(sizeStart-1, sizeEnd) :
        for i in range(sizeStart-1, sizeEnd) :
            aold[i,j] = anew[i,j] / 2.0 +  \
                        (  anew[i-1,j] + anew[i+1,j] +  \
                            anew[i,j-1] + anew[i,j+1]   ) / 4.0 / 2.0

    for i in range(0, nsources) :
        x = sources[i,0]
        y = sources[i,1]     
        aold[x, y] = aold[x, y] + energy

#END FOR 

heat = 0.0
for j in range(sizeStart-1, sizeEnd) :
    for i in range(sizeStart-1, sizeEnd) :
        heat = heat + aold[i,j]

aold = anew = None
t = time.time() - t
print("Heat = %0.4f | Tempo = %0.4f s" %(heat, t))

Heat = 30.0000 | Tempo = 0.2378 s


## Adicionando tipos e diretivas, otimizando
    Obs.: amarelo na listagem de saída significa que a parte do código tem interação com o interpretador e pode ser otimizado.

### Compiler directives
- **boundscheck (True / False)**: If set to False, Cython is free to assume that indexing operations ([]-operator) in the code will not cause any IndexErrors to be raised. Lists, tuples, and strings are affected only if the index can be determined to be non-negative (or if wraparound is False). Conditions which would normally trigger an IndexError may instead cause segfaults or data corruption if this is set to False. Default is True.
- **wraparound (True / False)**: In Python, arrays and sequences can be indexed relative to the end. For example, A[-1] indexes the last value of a list. In C, negative indexing is not supported. If set to False, Cython is allowed to neither check for nor correctly handle negative indices, possibly causing segfaults or data corruption. If bounds checks are enabled (the default, see boundschecks above), negative indexing will usually raise an IndexError for indices that Cython evaluates itself. However, these cases can be difficult to recognise in user code to distinguish them from indexing or slicing that is evaluated by the underlying Python array or sequence object and thus continues to support wrap-around indices. It is therefore safest to apply this option only to code that does not process negative indices at all. Default is True.
- **cdivision (True / False)**: If set to False, Cython will adjust the remainder and quotient operators C types to match those of Python ints (which differ when the operands have opposite signs) and raise a ZeroDivisionError when the right operand is 0. This has up to a 35% speed penalty. If set to True, no checks are performed. See CEP 516. Default is False.
- **initializedcheck (True / False)**: If set to True, Cython checks that a memoryview is initialized whenever its elements are accessed or assigned to. Setting this to False disables these checks. Default is True.
- **language_level (2/3/3str)**: Globally set the Python language level to be used for module compilation. Default is compatibility with Python 2. To enable Python 3 source code semantics, set this to 3 (or 3str) at the start of a module or pass the “-3” or “–3str” command line options to the compiler. The 3str option enables Python 3 semantics but does not change the str type and unprefixed string literals to unicode when the compiled code runs in Python 2.x. Note that cimported files inherit this setting from the module being compiled, unless they explicitly set their own language level. Included source files always inherit this setting.
- **infer_types (True / False)**: Infer types of untyped variables in function bodies. Default is None, indicating that only safe (semantically-unchanging) inferences are allowed. In particular, inferring integral types for variables used in arithmetic expressions is considered unsafe (due to possible overflow) and must be explicitly requested.

Fonte: https://cython.readthedocs.io/en/latest/src/userguide/source_files_and_compilation.html

In [2]:
%reload_ext Cython

In [5]:
%%cython?

Docstring:
::

  %cython [-a] [-+] [-3] [-2] [-f] [-c COMPILE_ARGS]
              [--link-args LINK_ARGS] [-l LIB] [-n NAME] [-L dir] [-I INCLUDE]
              [-S SRC] [--pgo] [--verbose]

Compile and import everything from a Cython code cell.

The contents of the cell are written to a `.pyx` file in the
directory `IPYTHONDIR/cython` using a filename with the hash of the
code. This file is then cythonized and compiled. The resulting module
is imported and all of its symbols are injected into the user's
namespace. The usage is similar to that of `%%cython_pyximport` but
you don't have to pass a module name::

    %%cython
    def f(x):
        return 2.0*x

To compile OpenMP codes, pass the required  `--compile-args`
and `--link-args`.  For example with gcc::

    %%cython --compile-args=-fopenmp --link-args=-fopenmp
    ...

To enable profile guided optimisation, pass the ``--pgo`` option.
Note that the cell itself needs to take care of establishing a suitable
profile when executed. 

### Testes

In [3]:
%%cython --force --compile-args=-O3
#cython: boundscheck=False, wraparound=False, cdivision=True
#cython: initializedcheck=False, language_level=3, infer_types=True
#  %%cython --annotate

import sys
import time
import numpy as np

cdef int n, niters, nsources, x, y
cdef double energy, heat, t

# array indices
cdef Py_ssize_t iters, i, j, size, sizeStart, sizeEnd

# inicialização de parâmetros
# 4800, 1, 500 = 1500; 100, 1, 10 = 30
n      = 4800   # nxn grid
energy = 1.0    # energy to be injected per iteration
niters = 5    # number of iterations

# incializa variáveis
iters = i = j = size = sizeStart = sizeEnd = x = y = 0  # int(4)
size = n + 2
sizeStart = 2
sizeEnd = n + 1

# cria e inicializa as matrizes com zeros e memoryview
cdef double[:,::1] mvaold = np.zeros((size, size), np.double)
cdef double[:,::1] mvanew = np.zeros((size, size), np.double)
nsources = 3
cdef    int[:,::1] mvsources = np.empty( (nsources,2), np.intc)

# inicializa as 3 fontes de calor
mvsources[0,0] = mvsources[0,1] = n/2
mvsources[1,0] = mvsources[1,1] = n/3
mvsources[2,0] = n*4/5
mvsources[2,1] = n*8/9

heat = 0.0      # calor total no sistema
t = time.time()
for iters in range(0, niters, 2) :
    ### iteracao impar: anew <- stencil(aold) ###
    # indices em C começam em 0, e em fortran começam em 1
    # range em python termina antes do fim: é "<" e não "<="
    for j in range(sizeStart-1, sizeEnd) :
        for i in range(sizeStart-1, sizeEnd) :
            mvanew[i,j] =  mvaold[i,j] / 2.0 + \
                         ( mvaold[i-1,j] + mvaold[i+1,j] + \
                           mvaold[i,j-1] + mvaold[i,j+1] ) / 4.0 / 2.0
    # indices precisam ser subtraídos de 1 porque em python \
    # começam em 0
    for i in range(0, nsources) :
        x = mvsources[i,0]
        y = mvsources[i,1]
        mvanew[x, y] = mvanew[x, y] + energy
    ### iteracao par: aold <- stencil(anew) ###
    for j in range(sizeStart-1, sizeEnd) :
        for i in range(sizeStart-1, sizeEnd) :
            mvaold[i,j] =  mvanew[i,j] / 2.0 + \
                         ( mvanew[i-1,j] + mvanew[i+1,j] + \
                           mvanew[i,j-1] + mvanew[i,j+1] ) / 4.0 / 2.0
    for i in range(0, nsources) :
        x = mvsources[i,0]
        y = mvsources[i,1]     
        mvaold[x, y] = mvaold[x, y] + energy
for j in range(sizeStart-1, sizeEnd) :
    for i in range(sizeStart-1, sizeEnd) :
        heat = heat + mvaold[i,j]

t = time.time() - t
print("Heat = %0.4f | Tempo = %0.4f" %(heat, t))

Heat =   18.0000 | Tempo =   5.1714


### Testes

In [5]:
%%cython  --force  --compile-args=-O3  --annotate
#cython: boundscheck=False, wraparound=False, cdivision=True
#cython: initializedcheck=False, language_level=3, infer_types=True
# %%cython 

from time import time
import numpy as np

# inicialização de parâmetros
# 4800, 1, 500 = 1500; 4800, 1, 5 = 18
cdef Py_ssize_t  n      = 4800   # nxn grid
cdef double      energy = 1.0    # energia a ser inserida
cdef Py_ssize_t  niters = 500    # qde de interacoes

# definição de variáveis
cdef double      heat      = 0.0
cdef double      t         = 0.0
cdef Py_ssize_t  size      = n + 2
cdef Py_ssize_t  sizeStart = 1
cdef Py_ssize_t  sizeEnd   = n + 1
cdef Py_ssize_t  iters, i, j

t=time()

# cria e inicializa as matrizes com zeros e memoryview
cdef double[:,::1] mvaold    = np.zeros((size, size), np.double)
cdef double[:,::1] mvanew    = np.zeros((size, size), np.double)
cdef Py_ssize_t    nsources  = 3      # qde de fontes
cdef int[:,::1]    mvsources = np.empty((nsources,2), np.intc)

# inicializa as 3 fontes de calor
mvsources[0,0] = mvsources[0,1] = n/2
mvsources[1,0] = mvsources[1,1] = n/3
mvsources[2,0] = n*4/5
mvsources[2,1] = n*8/9

niters = (niters + 1) // 2
t = time()
for iters in range(niters) :
    # iteracao impar
    for i in range(sizeStart, sizeEnd) :
        for j in range(sizeStart, sizeEnd) :
            mvanew[i,j] = ( mvaold[i,j] / 2.0 +
                          ( mvaold[i-1,j] + mvaold[i+1,j] +
                            mvaold[i,j-1] + mvaold[i,j+1] ) / 8.0 )
    for i in range(nsources) :
        mvanew[mvsources[i,0], mvsources[i,1]] += energy
    # iteracao par
    for i in range(sizeStart, sizeEnd) :
        for j in range(sizeStart, sizeEnd) :
            mvaold[i,j] = ( mvanew[i,j] / 2.0 +
                          ( mvanew[i-1,j] + mvanew[i+1,j] +
                            mvanew[i,j-1] + mvanew[i,j+1] ) / 8.0 )
    for i in range(nsources) :
        mvaold[mvsources[i,0], mvsources[i,1]] += energy
# calcula o total de energia
for i in range(sizeStart, sizeEnd) :
    for j in range(sizeStart, sizeEnd) :
        heat += mvaold[i,j]
t = time() - t

print("Heat = %0.4f | Tempo = %0.4f" %(heat, t))

Heat = 1500.0000 | Tempo = 20.1569


# Compilando em linha de comando
E preparando para executar em lote

### Testes

In [6]:
%%writefile scs.pyx
#cython: boundscheck=False, wraparound=False, cdivision=True
#cython: initializedcheck=False, language_level=3, infer_types=True

cpdef st(int n, double energy, int niters):
    from time import time
    import numpy as np

    # definição de variáveis
    cdef double      heat      = 0.0
    cdef double      t         = 0.0
    cdef Py_ssize_t  size      = n + 2
    cdef Py_ssize_t  sizeStart = 1
    cdef Py_ssize_t  sizeEnd   = n + 1
    cdef Py_ssize_t  iters, i, j

    t = time()
    
    # cria e inicializa as matrizes com zeros e memoryview
    cdef double[:,::1] mvaold = np.zeros((size, size), np.double)
    cdef double[:,::1] mvanew = np.zeros((size, size), np.double)
    cdef Py_ssize_t    nsources  = 3      # qde de fontes
    cdef    int[:,::1] mvsources = np.empty( (nsources,2), np.intc)

    # inicializa 3 fontes de calor
    mvsources[0,0] = mvsources[0,1] = n/2
    mvsources[1,0] = mvsources[1,1] = n/3
    mvsources[2,0] = n*4/5
    mvsources[2,1] = n*8/9

    niters = (niters + 1) // 2
    for iters in range(niters) :
        # iteracao impar
        for i in range(sizeStart, sizeEnd) :
            for j in range(sizeStart, sizeEnd) :
                mvanew[i,j] = ( mvaold[i,j] / 2.0 +
                              ( mvaold[i-1,j] + mvaold[i+1,j] +
                                mvaold[i,j-1] + mvaold[i,j+1] ) / 8.0 )
        for i in range(nsources) :
            mvanew[mvsources[i,0], mvsources[i,1]] += energy
        # iteracao par
        for i in range(sizeStart, sizeEnd) :
            for j in range(sizeStart, sizeEnd) :
                mvaold[i,j] = ( mvanew[i,j] / 2.0 +
                              ( mvanew[i-1,j] + mvanew[i+1,j] +
                                mvanew[i,j-1] + mvanew[i,j+1] ) / 8.0 )
        for i in range(nsources) :
            mvaold[mvsources[i,0], mvsources[i,1]] += energy
    # calcula o total de energia
    for i in range(sizeStart, sizeEnd) :
        for j in range(sizeStart, sizeEnd) :
            heat += mvaold[i,j]
    t = time() - t
#    print("Heat = %0.4f | Tempo = %0.4f" %(heat, t))
    return heat, t

Overwriting scs.pyx


In [7]:
%%writefile setup.py
from setuptools import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize("scs.pyx", force=True)
)

Overwriting setup.py


# Build
Intel Compiler: workaround to icc -no-gcc failing on new glibc system headers (need on Fedora 32):

```bash
LDSHARED="icc -shared" CC=icc CFLAGS="-D_Float32=__Float32 -D_Float32x=__Float32x -D_Float64=__Float64 -D_Float64x=__Float64x" python setup.py build_ext --inplace
```

Fonte: https://github.com/easybuilders/easybuild-easyconfigs/issues/10932

Para rodar no sdumont o workaround não é necessário:

In [8]:
%%bash
rm scs.*.so
source /scratch/app/modulos/intel-psxe-2019.sh
LDSHARED="icc -shared" CC=icc python setup.py build_ext --inplace

Intel(R) Parallel Studio XE 2019 Update 3 for Linux*
Copyright (C) 2009-2019 Intel Corporation. All rights reserved.
[1/1] Cythonizing scs.pyx
running build_ext
building 'scs' extension
icc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wformat -Wformat-security -D_FORTIFY_SOURCE=2 -fstack-protector -O3 -fpic -fPIC -Wformat -Wformat-security -D_FORTIFY_SOURCE=2 -fstack-protector -O3 -fpic -fPIC -fPIC -I/opt/intel/parallel_studio_xe_2019/intelpython3/include/python3.6m -c scs.c -o build/temp.linux-x86_64-3.6/scs.o
icc -shared build/temp.linux-x86_64-3.6/scs.o -L/opt/intel/parallel_studio_xe_2019/intelpython3/lib -lpython3.6m -o build/lib.linux-x86_64-3.6/scs.cpython-36m-x86_64-linux-gnu.so
copying build/lib.linux-x86_64-3.6/scs.cpython-36m-x86_64-linux-gnu.so -> 


icc: command line warning #10148: option '-Wno-unused-result' not supported


In [6]:
! ls -lh *.so

-rwxr-xr-x 1 xxxx.xxxx ampemi 562K Nov 10 14:19 scs.cpython-36m-x86_64-linux-gnu.so


In [13]:
%reload_ext autoreload
%autoreload 2
#reloads modules automatically before execution

import scs
print(scs.__doc__)

None


## Código Python que chama o módulo Cython

In [9]:
%%writefile st-cy-seq.py
from time import time
tp = time()
import scs

n            = 4800    # nxn grid; 4800,1,500→1500; 100,1,10→30 [4800]
energy       = 1.0     # energy to be injected per iteration [1.0]
niters       = 500     # number of iterations [500]

heat, t = scs.st(n, energy, niters)
tp = time() - tp
print("Heat = %0.4f | Tempo = %0.4f | TempoPyt = %0.4f" %(heat, t, tp))

Overwriting st-cy-seq.py


## Testando o funcionamento

In [10]:
%%bash
source /scratch/app/modulos/intel-psxe-2019.sh
python st-cy-seq.py

Intel(R) Parallel Studio XE 2019 Update 3 for Linux*
Copyright (C) 2009-2019 Intel Corporation. All rights reserved.
Heat = 1500.0000 | Tempo = 36.7287 | TempoPyt = 37.3191


#### Usando GCC e compararando a diferença

In [11]:
%%bash
rm scs.*.so
python setup.py build_ext --inplace

[1/1] Cythonizing scs.pyx
running build_ext
building 'scs' extension
gcc -pthread -B /scratch/app/anaconda3/2018.12/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/scratch/app/anaconda3/2018.12/include/python3.7m -c scs.c -o build/temp.linux-x86_64-3.7/scs.o
gcc -pthread -shared -B /scratch/app/anaconda3/2018.12/compiler_compat -L/scratch/app/anaconda3/2018.12/lib -Wl,-rpath=/scratch/app/anaconda3/2018.12/lib -Wl,--no-as-needed -Wl,--sysroot=/ build/temp.linux-x86_64-3.7/scs.o -o build/lib.linux-x86_64-3.7/scs.cpython-37m-x86_64-linux-gnu.so
copying build/lib.linux-x86_64-3.7/scs.cpython-37m-x86_64-linux-gnu.so -> 


In [12]:
%%bash
python st-cy-seq.py

Heat = 1500.0000 | Tempo = 19.2412 | TempoPyt = 19.4514


In [15]:
%%writefile calctempo.sh
#!/bin/sh
time python st-cy-seq.py

Overwriting calctempo.sh


In [16]:
%%bash
sh calctempo.sh

Heat = 1500.0000 | Tempo = 20.8582 | TempoPyt = 21.3222



real	0m21.413s
user	0m20.304s
sys	0m0.818s


Com GCC o tempo é menor. Adotanto o GCC então como padrão para Cython

## Rodando em um nó de execução

Caso necessário, é possível copiar o ambiente "envs" para /scratch

#### Copia o executável para /scratch :

In [17]:
%%bash
s='/prj/ampemi/xxxx.xxxx/stnc/Cython'
d='/scratch/ampemi/xxxx.xxxx/stnc/Cython'
rm $d/scs.*.so
cp  $s/scs.*.so  $s/st-cy-seq.py  $s/calctempo.sh  $d

Conferindo se funciona, antes da execução em lote:

In [18]:
%%bash
a='/scratch/ampemi/xxxx.xxxx'
d='/scratch/ampemi/xxxx.xxxx/stnc/Cython'
cd $d
sh calctempo.sh

Heat = 1500.0000 | Tempo = 21.1068 | TempoPyt = 21.3335



real	0m21.393s
user	0m20.228s
sys	0m1.126s


## Arquivo de submissão

In [19]:
%%writefile st-cy-seq.srm
#!/bin/bash
# limites das filas (1,0 UA):
#   cpu_dev  : 20 min.,  1-4  nós, 1/1   tarefas em exec/fila máximo
#   cpu_small: 72 horas, 1-20 nós, 16/96 tarefas em exec/fila máximo
#SBATCH -p cpu_small           #Fila (partition) a ser utilizada
#SBATCH --ntasks=1             #Total de tarefas
#SBATCH --nodes=1              #Qtd de nós
#SBATCH --ntasks-per-node=1    #Qtd de tarefas por nó ($SLURM_NTASKS_PER_NODE)
#SBATCH -J stcyseq             #Nome do job, 8 caracteres
# #SBATCH --exclusive            #Utilização exclusiva dos nós
#SBATCH --time=00:02:00        #Tempo max. de execução 2 minutos

echo '========================================'
echo '- Stencil Cython Sequencial'
echo '- Job ID:' $SLURM_JOB_ID
echo '- Tarefas por no:' $SLURM_NTASKS_PER_NODE
echo '- Qtd. de nos:' $SLURM_JOB_NUM_NODES
echo '- Tot. de tarefas:' $SLURM_NTASKS
echo '- Nos alocados:' $SLURM_JOB_NODELIST
echo '- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):'
echo $SLURM_SUBMIT_DIR
cd $SLURM_SUBMIT_DIR

#Entra no diretório de trabalho
cd /scratch/ampemi/xxxx.xxxx/stnc/Cython

#Executavel
EXEC='sh calctempo.sh'

#Dispara a execucao
echo '-- srun -------------------------------'
echo '$ time srun -n ' $SLURM_NTASKS $EXEC
srun -n $SLURM_NTASKS $EXEC
echo '-- FIM --------------------------------'

Overwriting st-cy-seq.srm


## Envia para execução 3 vezes

In [20]:
%%bash
sbatch st-cy-seq.srm
sbatch st-cy-seq.srm
sbatch st-cy-seq.srm
squeue -n stcyseq

Submitted batch job 772612
Submitted batch job 772613
Submitted batch job 772614
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            772612 cpu_small  stcyseq xxxx. PD       0:00      1 (Priority)
            772613 cpu_small  stcyseq xxxx. PD       0:00      1 (Priority)
            772614 cpu_small  stcyseq xxxx. PD       0:00      1 (Priority)


### Verifica se já terminou

In [25]:
%%bash
squeue -n stcyseq

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            772614 cpu_small  stcyseq xxxx. PD       0:00      1 (Resources)
            772613 cpu_small  stcyseq xxxx. PD       0:00      1 (Resources)
            772612 cpu_small  stcyseq xxxx. PD       0:00      1 (Resources)


## Mostra os arquivos de saída

In [1]:
%%bash
b='/stnc/Cython'
d='/scratch/ampemi/xxxx.xxxx'$b
cat $d/slurm-772612.out
cat $d/slurm-772613.out
cat $d/slurm-772614.out

- Stencil Cython Sequencial
- Job ID: 772612
- Tarefas por no: 1
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1407
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Cython
-- srun -------------------------------
$ time srun -n  1 sh calctempo.sh
Heat = 1500.0000 | Tempo = 23.9847 | TempoPyt = 29.4011

real	0m29.715s
user	0m24.100s
sys	0m0.335s
-- FIM --------------------------------
- Stencil Cython Sequencial
- Job ID: 772613
- Tarefas por no: 1
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1407
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Cython
-- srun -------------------------------
$ time srun -n  1 sh calctempo.sh
Heat = 1500.0000 | Tempo = 23.9444 | TempoPyt = 24.6716

real	0m24.811s
user	0m24.077s
sys	0m0.255s
-- FIM --------------------------------
- Stencil Cython Sequencial
- Job ID: 772614
- Tarefas por no: 1
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont14